## Exp 7: a validation script for Exp 7
### Result:


In [1]:
from __future__ import print_function
from __future__ import absolute_import
import matplotlib.pyplot as plt

import sugartensor as tf
import os
from IPython.display import display, Image
import numpy as np
from scipy import ndimage
from six.moves import cPickle as pickle

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
%matplotlib inline


In [2]:
batch_size = 1  # batch size
image_size = 64
pixel_depth = 255.0



## Validation data

In [3]:
dir= "barcode/valid/"
fn = os.listdir(dir)
total = len(fn)
total
#     change the range to switch between exp2 : [0, 1, 2, 3, 4, 5, 6]
#     and exp2.1: [1, 3, 5, 7, 9]
visible = range(15)
# put all the images into this blob of size total*size*size*1
# REMEMBER to change shape of dataset
valid_dataset = np.ndarray(shape = (3000*len(visible), image_size, image_size, 1), dtype = np.float32)
counter = 0
# REMEMBER to change shape of valid label
valid_label = np.ndarray(shape = (3000*len(visible)), dtype = np.float32)
for file in fn:
#     image_data = ndimage.imread(dir+file).astype(float)
    image_data = (ndimage.imread(dir+file).astype(float) - 255/2) / pixel_depth
    label = int(file.split("_")[0])
    if label in visible:
        valid_label[counter] = label
        valid_dataset[counter, :, :] = image_data[:,:,0].reshape(image_size, image_size, 1)
        counter+=1
#     else:
#         print(file[0])
# shuffle dataset
permutation = np.random.permutation(counter)
valid_dataset = valid_dataset[permutation,:,:,:]
valid_label = valid_label[permutation]

# Model

In [4]:
train_data_node = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, 1))

train_labels_node = tf.placeholder(tf.float32, shape =(batch_size, 1))

def print_variable():
    for v in tf.global_variables():
        print(v)
# capping stddev at 0.1 is important to avoid exploding value
def conv_relu(input, kernel_shape, bias_shape, stride = 2):
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides = [1, stride, stride, 1], padding = 'SAME')
    return tf.nn.relu(conv+biases)
def fully_connected(input, out_shape):
    input_shape = input.get_shape().as_list()
    batch = input_shape[0]
    flattened_size = input_shape[1]
    weights = tf.get_variable("weights", [flattened_size, out_shape], 
                              initializer=tf.truncated_normal_initializer(stddev=0.1))
    biases = tf.get_variable("biases", [out_shape],
                            initializer = tf.constant_initializer(0.1))
    return tf.matmul(input, weights) + biases
def network(input_images):
    with tf.variable_scope("conv1"):
        # kernel is in shape height*width*in*out
        conv_relu1 = conv_relu(input_images, [4, 4, 1, 32], [32])
    with tf.variable_scope("conv2"):
        conv_relu2 = conv_relu(conv_relu1, [4, 4, 32, 64], [64])
    with tf.variable_scope("conv3"):
        conv_relu3 = conv_relu(conv_relu2, [4, 4, 64, 128], [128])
    with tf.variable_scope("fc1"):
        conv_relu3_shape = conv_relu3.get_shape().as_list()
        reshape_conv3 = tf.reshape(conv_relu3,
                                [conv_relu3_shape[0], conv_relu3_shape[1]*conv_relu3_shape[2]*conv_relu3_shape[3]])
        fc1 = tf.nn.relu(fully_connected(reshape_conv3, 1024))
    with tf.variable_scope("lstm"):
        num_hidden = 30
        fc1_shape = fc1.get_shape().as_list()
        print(fc1_shape)
        fc1_reshape = tf.reshape(fc1, [fc1_shape[0], fc1_shape[1], 1])
        cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
        val, _ = tf.nn.dynamic_rnn(cell, fc1_reshape, dtype = tf.float32)
        print(val)
        val = tf.transpose(val, [1, 0, 2]) #transpose to easily get the last state
        last = tf.gather(val, int(val.get_shape()[0]) - 1)
        print(last)
        weight = tf.Variable(tf.truncated_normal([num_hidden, 1]))
        bias = tf.Variable(tf.constant(0.1, shape=[1]))
        output = tf.matmul(last, weight)+bias
    return output
    
output = network(train_data_node)

[1, 1024]
Tensor("lstm/rnn/transpose:0", shape=(1, 1024, 30), dtype=float32)
Tensor("lstm/Gather:0", shape=(1, 30), dtype=float32)


# Validate


In [5]:
saver = tf.train.Saver()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)

sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
sess.as_default()
saver.restore(sess, "./pureTensorflowExp/Exp7/19500.ckpt")


offset = 0
batch_data_all = valid_label
results = []
correct = 0
for i in range(len(valid_label)):
    batch_data = valid_dataset[offset+i:offset+i+1]
    batch_labels = valid_label[offset+i:offset+i+1].reshape((batch_size, 1))
    predictions = sess.run(output, feed_dict={train_data_node: batch_data})[0][0]
    predictions = float(int(round(predictions)))
    if predictions == batch_data_all[i]:
        correct+=1
    results.append(predictions)
print(batch_data_all[100:120])
print(results[100:120])
print("accuracy: "+str(correct/len(valid_label)))
confusions = np.zeros([10, 10], np.float32)

INFO:tensorflow:Restoring parameters from ./pureTensorflowExp/Exp7/19500.ckpt
[ 12.   1.  11.   9.   7.   0.   1.  13.   0.   7.   2.  12.  13.   7.   9.
   3.  10.   7.   6.   8.]
[9.0, 1.0, 9.0, 9.0, 7.0, 0.0, 1.0, 9.0, 0.0, 7.0, 2.0, 9.0, 9.0, 7.0, 9.0, 3.0, 9.0, 7.0, 6.0, 8.0]
accuracy: 0.5217777777777778
